# Word Embeddings: Ungraded Practice Notebook

In this ungraded notebook, you'll try out all the individual techniques that you learned about in the lecture. Practicing on small examples will prepare you for the graded assignment, where you will combine the techniques in more advanced ways to create word embeddings from a real-life corpus.

This notebook is made of two main parts: data preparation, and the continuous bag-of-words (CBOW) model.

To get started, import and initialize all the libraries you will need.

In [1]:
import os
import pickle
import re

import emoji
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

from utils import get_dict

#nltk.download('punkt') broken ... workaround below

In [2]:
HOME = os.environ['HOME']
punkt_path = f'{HOME}/nltk_data/tokenizers/punkt/PY3/english.pickle'

In [3]:
with open(punkt_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Data preparation

In the data preparation phase, starting with a corpus of text, you will:

- Clean and tokenize the corpus.

- Extract the pairs of context words and center word that will make up the training data set for the CBOW model. The context words are the features that will be fed into the model, and the center words are the target values that the model will learn to predict.

- Create simple vector representations of the context words (features) and center words (targets) that can be used by the neural network of the CBOW model.

## Cleaning and tokenization

To demonstrate the cleaning and tokenization process, consider a corpus that contains emojis and various punctuation signs.

In [23]:
corpus = 'Who ❤️ "word embeddings" in 2020? I do!!!'
print(f'Corpus:  {corpus}')
data = re.sub(r'[,!?;-]+', '.', corpus)
print(f'After cleaning punctuation:  {data}')

Corpus:  Who ❤️ "word embeddings" in 2020? I do!!!
After cleaning punctuation:  Who ❤️ "word embeddings" in 2020. I do.


In [24]:
print(f'Initial string:  {data}')
data = nltk.word_tokenize(data)
print(f'After tokenization:  {data}')

Initial string:  Who ❤️ "word embeddings" in 2020. I do.
After tokenization:  ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.']


In [25]:
print(f'Initial list of tokens:  {data}')
data = [ ch.lower() for ch in data
         if ch.isalpha()
         or ch == '.'
         or emoji.get_emoji_regexp().search(ch)
       ]
print(f'After cleaning:  {data}')

Initial list of tokens:  ['Who', '❤️', '``', 'word', 'embeddings', "''", 'in', '2020', '.', 'I', 'do', '.']
After cleaning:  ['who', '❤️', 'word', 'embeddings', 'in', '.', 'i', 'do', '.']


In [26]:
# wrap in func
def tokenize(corpus):
    data = re.sub(r'[,!?;-]+', '.', corpus)
    data = nltk.word_tokenize(data)
    data = [ch.lower() for ch in data 
            if ch.isalpha()
            or ch == '.'
            or emoji.get_emoji_regexp().search(ch)]
    return data

In [27]:
corpus = 'I am happy because I am learning'
print(f'Corpus:  {corpus}')
words = tokenize(corpus)
print(f'Words (tokens):  {words}')

Corpus:  I am happy because I am learning
Words (tokens):  ['i', 'am', 'happy', 'because', 'i', 'am', 'learning']


In [28]:
tokenize('A man, a plan, a canal: "Panama!"')

['a', 'man', '.', 'a', 'plan', '.', 'a', 'canal', 'panama', '.']

## Sliding window of words
Now that you have transformed the corpus into a list of clean tokens, you can slide a window of words across this list. For each window you can extract a center word and the context words.

The `get_windows` function in the next cell was introduced in the lecture.

In [30]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i + 1):(i + C + 1)]
        yield context_words, center_word
        i += 1

The first argument of this function is a list of words (or tokens). The second argument, `C`, is the context half-size. Recall that for a given center word, the context words are made of `C` words to the left and `C` words to the right of the center word.

Here is how you can use this function to extract context words and center words from a list of tokens. These context and center words will make up the training set that you will use to train the CBOW model.

In [31]:
for x, y in get_windows(
    ['i', 'am', 'happy', 'because', 'i', 'am', 'learning'], 2):
    print(f'{x}\t{y}')

['i', 'am', 'because', 'i']	happy
['am', 'happy', 'i', 'am']	because
['happy', 'because', 'am', 'learning']	i


The first example of the training set is made of:

- the context words "i", "am", "because", "i",

- and the center word to be predicted: "happy".

**Now try it out yourself. In the next cell, you can change both the sentence and the context half-size.**

In [33]:
for x, y in get_windows(
    tokenize(
        "I'm a lumberjack and I'm ok, I sleep all night and I work all day"), 
    3):
    print(f'{x}\t{y}')

['i', 'a', 'lumberjack', 'i', 'ok', '.']	and
['a', 'lumberjack', 'and', 'ok', '.', 'i']	i
['lumberjack', 'and', 'i', '.', 'i', 'sleep']	ok
['and', 'i', 'ok', 'i', 'sleep', 'all']	.
['i', 'ok', '.', 'sleep', 'all', 'night']	i
['ok', '.', 'i', 'all', 'night', 'and']	sleep
['.', 'i', 'sleep', 'night', 'and', 'i']	all
['i', 'sleep', 'all', 'and', 'i', 'work']	night
['sleep', 'all', 'night', 'i', 'work', 'all']	and
['all', 'night', 'and', 'work', 'all', 'day']	i


## Transforming words into vectors for the training set
To finish preparing the training set, you need to transform the context words and center words into vectors.

### Mapping words to indices and indices to words

The center words will be represented as one-hot vectors, and the vectors that represent context words are also based on one-hot vectors.

To create one-hot word vectors, you can start by mapping each unique word to a unique integer (or index). We have provided a helper function, `get_dict`, that creates a Python dictionary that maps words to integers and back.

In [34]:
word2ind, ind2word = get_dict(words)

In [35]:
word2ind

{'am': 0, 'because': 1, 'happy': 2, 'i': 3, 'learning': 4}

In [37]:
print("Index of the word 'i':  ", word2ind['i'])

Index of the word 'i':   3


In [38]:
ind2word

{0: 'am', 1: 'because', 2: 'happy', 3: 'i', 4: 'learning'}

In [40]:
print("Word which has index 2:", ind2word[2] )

Word which has index 2: happy


In [42]:
V = len(word2ind)
print("Size of vocabulary:", V)

Size of vocabulary: 5


### Getting one-hot word vectors

Recall from the lecture that you can easily convert an integer, $n$, into a one-hot vector.

Consider the word "happy". First, retrieve its numeric index.

In [43]:
n = word2ind['happy']
n

2

In [44]:
center_word_vector = np.zeros(V)
center_word_vector

array([0., 0., 0., 0., 0.])

In [45]:
len(center_word_vector) == V

True

In [47]:
center_word_vector[n] = 1
center_word_vector

array([0., 0., 1., 0., 0.])

In [48]:
def word_to_one_hot_vector(word, word2ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2ind[word]] = 1
    return one_hot_vector

In [49]:
word_to_one_hot_vector('happy', word2ind, V)

array([0., 0., 1., 0., 0.])

In [50]:
word_to_one_hot_vector('learning', word2ind, V)

array([0., 0., 0., 0., 1.])

### Getting context word vectors
To create the vectors that represent context words, you will calculate the average of the one-hot vectors representing the individual words.

Let's start with a list of context words.

In [51]:
context_words = ['i', 'am', 'because', 'i']

Using Python's list comprehension construct and the `word_to_one_hot_vector` function that you created in the previous section, you can create a list of one-hot vectors representing each of the context words.

In [52]:
context_words_vectors = [word_to_one_hot_vector(w, word2ind, V) 
                         for w in context_words]
context_words_vectors

[array([0., 0., 0., 1., 0.]),
 array([1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0.]),
 array([0., 0., 0., 1., 0.])]

And you can now simply get the average of these vectors using numpy's `mean` function, to get the vector representation of the context words.

In [53]:
np.mean(context_words_vectors, axis=0)

array([0.25, 0.25, 0.  , 0.5 , 0.  ])

Note the `axis=0` parameter that tells `mean` to calculate the average of the rows (if you had wanted the average of the columns, you would have used `axis=1`).

**Now create the `context_words_to_vector` function that takes in a list of context words, a word-to-index dictionary, and a vocabulary size, and outputs the vector representation of the context words.**